<a href="https://colab.research.google.com/github/rexad/NLP_notebooks/blob/master/Kaggle/jigsaw/SimpleBiderectionalLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from tensorflow.keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.preprocessing import sequence, text
from gensim.models import KeyedVectors


import gensim.downloader as api
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors

import os.path
from google.colab import drive

In [3]:
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
'Colab Notebooks'   Colab_Notebooks   Data   ML_Training


In [4]:
base_path_embed = '/content/gdrive/My Drive/Data/Embeddings/'
embeddings = [
              'fasttext-wiki-news-subwords-300',
              'glove-twitter-100',
              'glove-wiki-gigaword-300'
              ]
for embed in embeddings:
  if os.path.isfile(base_path_embed+embed) == False:
    embed_file = api.load(embed)
    embed_file.save(base_path_embed+embed)
    del embed_file

In [5]:

#hyper-parameter 
num_nodes = 128
batch_size = 256
num_unrolling = 50
dropout = 0.2
MAX_LEN = 220
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = num_nodes  * 4

In [6]:




identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness'
]

def build_model(matrix_embed):
  words = Input(shape=(None,))

  x = Embedding(*matrix_embed.shape, weights=[matrix_embed], trainable=False)(words)
  x = Bidirectional(LSTM(num_nodes, return_sequences=True))(x)
  
  hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
  
  hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
  result = Dense(1, activation='sigmoid')(hidden)

  model = Model(inputs=words, outputs=result)
  model.compile(loss='binary_crossentropy', optimizer='adam')
  return model


def build_embed_matrix(embed, tokens):

  embed_vec = KeyedVectors.load(base_path_embed + embed, mmap='r')
  embed_matrix = np.zeros((len(tokens) + 1,300))
  for i, word in tokens.items():
    for candidate in [word, word.lower()]:
      if candidate in embed_vec:
        embed_matrix[i]= embed_vec[candidate]
  return embed_matrix





In [7]:
#load code 

train_df = pd.read_csv('/content/gdrive/My Drive/Data/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test_df = pd.read_csv('/content/gdrive/My Drive/Data/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
tokenizer = text.Tokenizer(
    num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True)

aux_columns = ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']

x_train = train_df['comment_text'].astype(str)
y_train = train_df['target'].values
y_aux_train = train_df[aux_columns].values

x_test = test_df['comment_text'].astype(str)
tokenizer.fit_on_texts(list(x_train) + list(x_test))

for column in  identity_columns + ['target']:
    train_df[column] = np.where(train_df[column]>= 0.5, True, False)
    
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
x_train = sequence.pad_sequences(x_train, maxlen=MAX_LEN)
x_test = sequence.pad_sequences(x_test, maxlen=MAX_LEN)

sample_weights = np.ones(len(x_train), dtype=np.float32)
sample_weights += train_df[identity_columns].sum(axis=1)
sample_weights += train_df['target'] * (~train_df[identity_columns]).sum(axis=1)
sample_weights += (~train_df['target']) * train_df[identity_columns].sum(axis=1) * 5
sample_weights /= sample_weights.mean()



In [8]:
embed_matrix = build_embed_matrix('fasttext-wiki-news-subwords-300', tokenizer.index_word)

model = build_model(embed_matrix)

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=1,
    verbose=2
)
predictions = model.predict(x_test, batch_size=2048)
submission = pd.DataFrame({'id': test_df.id, 'prediction': list(predictions.flatten())})


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


7051/7051 - 1298s - loss: 0.2412


In [9]:
submission

,id,prediction
0,7097320,0.023798
1,7097321,0.099953
2,7097322,0.160539
3,7097323,0.053483
4,7097324,0.012230
...,...,...
97315,7194635,0.043183
97316,7194636,0.082581
97317,7194637,0.139601
97318,7194638,0.283837
